In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
!pip install vit_pytorch
from vit_pytorch import ViT
import torch.optim as optim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Defining the parameters
batch_size = 2
img_size = 32
patch_size = 4
dim = 64
depth = 6
heads = 8
mlp_dim = 128
dropout = 0.1
lr = 1e-3
epochs = 1

In [ ]:
# Data Augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(img_size,padding=2),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

In [ ]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
trainset = torchvision.datasets.MNIST(root='./data',train = True, download = True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data',train = False, download = True,transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 156144695.66it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 38321953.12it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 40510141.73it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 17721422.11it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Iterating over the datasets using dataloaders
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True)

In [ ]:
# ViT model
model = ViT(
    image_size=img_size,
    patch_size=patch_size,
    num_classes=10,
    dim = dim,
    depth=depth,
    heads=heads,
    mlp_dim=mlp_dim,
    dropout=dropout,   
    emb_dropout = 0.1,
    channels = 1
).to(device)

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = lr)

In [ ]:
# Training
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_correct = 0
    for batch_idx, (data,target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output.data, 1)
        train_correct += (predicted == target).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_acc = 100. * train_correct / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{epochs} Training Loss: {train_loss:.6f}, Training Accuracy: {train_acc:.2f}%')
    
    

Epoch 1/1 Training Loss: 0.419263, Training Accuracy: 86.64%
